In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score, mean_squared_error
os.chdir('/home/FRACTAL/ankush.raut/Downloads/Lecture assignments/Class 1 Module 2 data')

In [2]:
train = pd.read_csv('processed_train.csv')
test = pd.read_csv('processed_test.csv')
submission = pd.read_csv('sample_submission.csv')

In [4]:
pd.set_option('display.max_columns', None)

In [7]:
corr_matrix = train.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
correlated_columns = [column for column in upper.columns if any(upper[column] > 0.8)]

## Polynomial Features

In [9]:
from sklearn.preprocessing import PolynomialFeatures

In [43]:
polynomial_features = PolynomialFeatures(degree=2, interaction_only=True, include_bias=True)

In [16]:
training = train[['Id']+correlated_columns]
target = train[['SalePrice']]

test_ids = test[['Id']]

In [17]:
from sklearn.model_selection import train_test_split

In [44]:
xtrain, xtest, ytrain, ytest = train_test_split(training, target, test_size = 0.2, random_state = 0)

In [45]:
# xtrain = polynomial_features.fit_transform(xtrain.drop(columns = ['Id']))
# xtest = polynomial_features.transform(xtest.drop(columns = ['Id']))

In [38]:
xtrain_ids = xtrain[['Id']]
xtest_ids = xtest[['Id']]

xtrain = xtrain.drop(columns = ['Id'])
xtest = xtest.drop(columns = ['Id'])

In [46]:
from sklearn.linear_model import LinearRegression

In [47]:
lm_regressor = LinearRegression(fit_intercept=True, copy_X=True, n_jobs=-1)
lm_regression = lm_regressor.fit(xtrain, ytrain)

In [48]:
lm_reg_cv_score = np.mean(cross_val_score(lm_regression, xtrain, ytrain, cv = 5))
print('Linear Regression cross validated score =', lm_reg_cv_score)

Linear Regression cross validated score = 0.6689973210911131


In [49]:
lm_predictions = lm_regression.predict(xtest)
lm_r2 = r2_score(ytest.SalePrice, lm_predictions)
lm_rmse = np.sqrt(mean_squared_error(ytest.SalePrice, lm_predictions))

print('Linear Regression R^2 on hold out set =', lm_r2)
print('Linear Regression rmse on hold out set =', lm_rmse)
print('Mean SalePrice =', np.mean(ytest.SalePrice))

Linear Regression R^2 on hold out set = 0.35467343094833825
Linear Regression rmse on hold out set = 66757.23529068562
Mean SalePrice = 181370.38356164383


## PCA

In [50]:
from sklearn.decomposition import PCA

In [65]:
xtrain, xtest, ytrain, ytest = train_test_split(training, target, test_size = 0.2, random_state = 0)

In [66]:
pca = PCA(n_components=3, iterated_power='auto', random_state=0)

In [67]:
xtrain = pca.fit_transform(xtrain.drop(columns = ['Id']))
xtest = pca.transform(xtest.drop(columns = ['Id']))

In [68]:
lm_regressor = LinearRegression(fit_intercept=True, copy_X=True, n_jobs=-1)
lm_regression = lm_regressor.fit(xtrain, ytrain)

In [69]:
lm_reg_cv_score = np.mean(cross_val_score(lm_regression, xtrain, ytrain, cv = 5))
print('Linear Regression cross validated score =', lm_reg_cv_score)

Linear Regression cross validated score = 0.5358032119551579


In [70]:
lm_predictions = lm_regression.predict(xtest)
lm_r2 = r2_score(ytest.SalePrice, lm_predictions)
lm_rmse = np.sqrt(mean_squared_error(ytest.SalePrice, lm_predictions))

print('Linear Regression R^2 on hold out set =', lm_r2)
print('Linear Regression rmse on hold out set =', lm_rmse)
print('Mean SalePrice =', np.mean(ytest.SalePrice))

Linear Regression R^2 on hold out set = 0.3834206973260078
Linear Regression rmse on hold out set = 65253.38424309142
Mean SalePrice = 181370.38356164383
